# Mini Prompting Pipeline for Inference: InternVL2-Llama3-76B

following the quick start here:
https://huggingface.co/OpenGVLab/InternVL2-Llama3-76B#quick-start
or here:
https://internvl.readthedocs.io/en/latest/internvl2.0/quick_start.html

Structure:
1. Split, load (and save) the model (on two 80GB GPUs)
2. Preprocess images
3. Mini pipeline taking images from a folder and inputting them into the model with the same prompt
4. Playground for video input

## 0. Preparations

In [1]:
# installs
!pip install transformers==4.37.2
!pip install timm
!pip install accelerate
!pip install bitsandbytes
!pip install decord
!pip install pandas
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.6 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 144.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 142.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 171.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 63.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 155.1 MB/s eta 0:00:00


In [2]:
# packages
import torch
import numpy as np
import torchvision.transforms as T
from PIL import Image
import os
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import math

## 1. Split and load the model

In [3]:
# split the model on 2 80GB GPUs

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    num_layers = {
        'InternVL2-1B': 24, 'InternVL2-2B': 24, 'InternVL2-4B': 32, 'InternVL2-8B': 32,
        'InternVL2-26B': 48, 'InternVL2-40B': 60, 'InternVL2-Llama3-76B': 80}[model_name]
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

In [4]:
# load the splitted model

path = "OpenGVLab/InternVL2-Llama3-76B"

device_map = split_model('InternVL2-Llama3-76B')

model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-Llama3-76B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-Llama3-76B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
'''
# save the model

model_save_name = "InternVL2-Llama3-76B.pt"
model_path = "..."

torch.save(model.state_dict(), model_path)

# load the saved model
model.load_state_dict(torch.load(model_path, weights_only=True))
model.eval()
'''

In [5]:
# put model in evaluation mode

model.eval()

InternVLChatModel(
  (vision_model): InternVisionModel(
    (embeddings): InternVisionEmbeddings(
      (patch_embedding): Conv2d(3, 3200, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InternVisionEncoder(
      (layers): ModuleList(
        (0-44): 45 x InternVisionEncoderLayer(
          (attn): InternAttention(
            (qkv): Linear8bitLt(in_features=3200, out_features=9600, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (q_norm): InternRMSNorm()
            (k_norm): InternRMSNorm()
            (proj): Linear8bitLt(in_features=3200, out_features=3200, bias=True)
          )
          (mlp): InternMLP(
            (act): GELUActivation()
            (fc1): Linear8bitLt(in_features=3200, out_features=12800, bias=True)
            (fc2): Linear8bitLt(in_features=12800, out_features=3200, bias=True)
          )
          (norm1): InternRMSNorm()
          (norm2): InternRMSNorm()
  

## 2. Preprocessing of the images

In [7]:
# functions for preprocessing the input image

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

## 3. Generate Predictions

In [ ]:
# generate one example prediction

# load image, set the max number of tiles in `max_num`
pixel_values = load_image("/images/image1.jpg", max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# give 1 image and text as chat input (single-image single-round conversation) (find good prompt wording, insert template from LEIZA experts)
question = '<image>\nPretend to be an archivist who wants to catalog this photo card digitally. Write a description including different fields which are provided below. Also use the text on the photo card. ...'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'Assistant: \n {response}')

In [20]:
# mini pipeline: generate predictions for all images in the folder and save it in csv

# prompt
question = '<image>\nPretend to be an archivist who wants to catalog this photo card digitally. Write a description including different fields which are provided below. Also use the text on the photo card. ...'

# images folder
image_folder = "/images"

# dataframe to store image, prompt and response
responses = pd.DataFrame(columns=['image', 'prompt', 'response'])

i = 0

#print(question)

for filename in os.listdir(image_folder):
  if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):  # Add other extensions if needed

    # open image from folder
    image_path = os.path.join(image_folder, filename)

    # load image, set the max number of tiles in `max_num`
    pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
    generation_config = dict(max_new_tokens=1024, do_sample=True, temperature=0.01) # play around with temperature, num_beam and top_k

    # give image and text as chat input: single-image single-round conversation
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    print(f'Image: {i}, {filename}\nAssistant: {response}')
    responses = pd.concat([responses, pd.DataFrame({'image': [filename], 'prompt': [question], 'response': [response]})], ignore_index=True)
    i += 1  

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


Image: 0, BK_R16_00011.jpg
Assistant: Object title: Fendoch, Perthshire, Scotland
Object type: Black and white photograph
Technique/material: Photographic print
Motif classification: Architecture
Detailed motif classification: Military complex
Detailed description: Black and white photograph of a military complex in Fendoch, Perthshire, Scotland. The complex consists of several long, rectangular buildings arranged in a grid pattern, with a larger building at the center. The buildings appear to be barracks or similar military structures.
Additional information: 
- FO: Fendoch
- Fdst: Not specified
- Kreis: Perthshire
- Land: Scotland
- Zeit: Not specified
- Mus: Not specified
- Lit: Richmond-McIntyre, Proceedings of the soc. of Antiq. of Scotland, 1930/39, 110 ff.
Photo details: 
- Photographer and origin of photo unknown
- 1 photo mounted on photo card
- Card labelled by typewriter
- Photo discoloured yellow
Number of photos: 1
Image number: Not specified


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


Image: 1, BK_R16_00012.jpg
Assistant: Object title: View of the Roman London Wall
Object type: Colour photography
Technique/material: Photographic print
Motif classification: Architecture
Detailed motif classification: Fortification
Detailed description: Colour photograph of the Roman London Wall, showing a section of the wall with a guard tower and a river in the foreground.
Additional information: 
Photo details: Photographer and origin of photo unknown; 1 photo mounted on photo card; card labelled by hand; photo discoloured yellow; 
Number of photos: 1
Image number: not specified


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


...

Image: 7, BK_V85_Latene_L-Z_00067.jpg
Assistant: Object title: Landscape with water and land
Object type: Black and white photograph
Technique/material: Photographic print
Motif classification: Settlements and landscapes
Detailed motif classification: Landscape
Detailed description: Black and white photograph of a landscape featuring water and land. The water appears calm, and the land has some structures or buildings visible in the distance.
Additional information: Not specified
Photo details: Photographer and origin of photo unknown; 1 photo mounted on photo card; card labelled by hand; photo discoloured yellow; 
Number of photos: 1
Image number: not specified


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


Image: 8, BK_V85_Latene_L-Z_00068.jpg
Assistant: Object title: Excavation site with workers
Object type: Black and white photography
Technique/material: Photographic print
Motif classification: People
Detailed motif classification: Group picture
Detailed description: Black and white photograph of an excavation site with workers. The workers are seen digging and moving earth, with tools and equipment scattered around the site. The background shows a body of water, possibly a river or lake.
Additional information: 
Photo details: Photographer and origin of photo unknown; 1 photo mounted on photo card; card labelled by hand; photo discoloured yellow; 
Number of photos: 1
Image number: not specified


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


Image: 9, BK_R28_00002.jpg
Assistant: Object title: Fragment of a bronzeplastic vessel with a hand holding a bottle
Object type: Black and white photography
Technique/material: Photographic print
Motif classification: Object
Detailed motif classification: Sculpture
Detailed description: Black and white photograph of a fragment of a bronzeplastic vessel featuring a hand holding a bottle.
Additional information: 
- FO: Bregenz
- Fdst: not specified
- Kreis: not specified
- Land: Österreich
- Mus: Vorarlberger Landesmuseum Bregenz
- Zeit: not specified
- Lit: not specified
- Neg: not specified
Photo details: 
- Photographer and origin of photo unknown
- 1 photo mounted on photo card
- Card labelled by hand
- Photo discoloured yellow
Number of photos: 1
Image number: not specified


In [21]:
# save model in- and outputs in csv
responses_path = "/responses"
responses.to_csv(responses_path + "responses.csv")
responses.head()

,image,prompt,response
0,BK_R16_00011.jpg,<image>\nPretend to be an archivist who wants ...,"Object title: Fendoch, Perthshire, Scotland\nO..."
1,BK_R16_00012.jpg,<image>\nPretend to be an archivist who wants ...,Object title: View of the Roman London Wall\nO...
2,BK_V85_Latene_L-Z_00072.jpg,<image>\nPretend to be an archivist who wants ...,Object title: Iron Age weapons and shields\nOb...
3,Testobjekt_template_chat_GPT.jpg,<image>\nPretend to be an archivist who wants ...,Object title: Tree bark\nObject type: Black an...
4,BK_Bestimmung_06202300007.jpg,<image>\nPretend to be an archivist who wants ...,Object title: Arch of Constantine\nObject type...


---
---

## 4. Playground for video input

In [20]:
# video multi-round conversation

from decord import VideoReader, cpu

def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list

# load the video
video_path = '/video01.avi'
pixel_values, num_patches_list = load_video(video_path, num_segments=8, max_num=1)
pixel_values = pixel_values.to(torch.bfloat16).cuda()
video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])

In [23]:
question = video_prefix + 'What is the man doing?'
# Frame1: <image>\nFrame2: <image>\n...\nFrame8: <image>\n{question}

response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'Describe what happens in the video.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


User: Frame1: <image>
Frame2: <image>
Frame3: <image>
Frame4: <image>
Frame5: <image>
Frame6: <image>
Frame7: <image>
Frame8: <image>
What is the man playing?
Assistant: The man is playing basketball.


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


User: Pretend to be a human. Try to detect event boundaries in this video.
Assistant: The video shows a man playing basketball in a gym. There are no clear event boundaries in the video,
